In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install roboflow
!pip install ultralytics==8.3.40
!pip install streamlit ngrok opencv-python-headless
!pip install pyngrok
# !ngrok config add-authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

from detectron2.utils.logger import setup_logger
setup_logger()

from google.colab import drive
drive.mount('/content/gdrive')
ROOT_DIR = '/content/gdrive/My Drive/Georgios_Karagiannis/Code'

In [ ]:
# from roboflow import Roboflow

# rf = Roboflow(api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
# project = rf.workspace("------------------").project("------------------")
# version = project.version(2)
# dataset = version.download("coco")

# Simulation of the smart fridge

In [ ]:
%%writefile smart_fridge_app.py
import streamlit as st
import numpy as np
import glob
import cv2
import random
from google.colab.patches import cv2_imshow
import os
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from ultralytics import YOLO
import json

def register_coco_instances_with_classes(dataset_name, json_file, image_dir):
    register_coco_instances(dataset_name, {}, json_file, image_dir)

    with open(json_file, 'r') as f:
        coco_data = json.load(f)

    thing_classes = [category['name'] for category in coco_data['categories']]

    metadata = MetadataCatalog.get(dataset_name)
    metadata.thing_classes = thing_classes

register_coco_instances_with_classes("my_dataset_train", "/content/level1_images-2/train/_annotations.coco.json", "/content/level1_images-2/train")

def setup_faster_rcnn():
    cfg = get_cfg()
    cfg.merge_from_file("/content/gdrive/MyDrive/Georgios_Karagiannis/Code/config1.yml")
    cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/D-level1/model_final.pth"
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
    predictor = DefaultPredictor(cfg)
    metadata = MetadataCatalog.get("my_dataset_train")
    return predictor, metadata

def setup_yolo():
    model = YOLO("/content/gdrive/My Drive/Georgios_Karagiannis/Code/Level2_train/detect/train/weights/best.pt")
    return model

def analyze_image(image, temperature, humidity, predictor, metadata, yolo_model):
    im = cv2.imread(image)
    outputs = predictor(im)
    pred_classes_indices = outputs["instances"].pred_classes.cpu().numpy()
    pred_classes_names = [metadata.thing_classes[i] for i in pred_classes_indices]

    if "Tomato" in pred_classes_names:
        yolo_results = yolo_model(image)
        ripeness_classes = [yolo_model.names[int(cls)] for cls in yolo_results[0].boxes.cls]
        ripeness_message = []

        for ripeness in ripeness_classes:
            if ripeness == "over-ripe":
                ripeness_message.append("Over-ripe tomatoes detected. Discard them immediatley before they start to effect the other fruits.")
            elif ripeness == "ripe":
                if 7 <= temperature <= 10 and humidity > 70:
                    ripeness_message.append("Ripe tomatoes detected. They are edible for up to 4 days. If you want your tomatoes to last longer, you should store them somewhere cooler.")
                elif temperature < 7:
                    ripeness_message.append("You should put your ripe tomatoes somewhere warmer to avoid chilling injuries.")
                else:
                    ripeness_message.append("YYour tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
            elif ripeness == "half-ripe":
                if 8 <= temperature <= 10 and humidity > 60:
                    ripeness_message.append("Half-ripe tomatoes detected. They are not edible but can be stored for up to 7 days untill they are edible.If you want to ripe them more slower , you should store them somewhere cooler and with less humidity")
                elif temperature < 8:
                    ripeness_message.append("Your rippening prosses will be stoped and your tomatoes will be susceptible to chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
            elif ripeness == "unripe":
                if 8 <= temperature <= 12 and humidity > 50:
                    ripeness_message.append("Unripe tomatoes detected. They are not edible but can be stored for up to 10 days untill they are edible.If you want to ripe them more slower , you should store them somewhere cooler and with less humidity")
                elif temperature < 8:
                    ripeness_message.append("Your rippening prosses will be stoped and your tomatoes will be susceptible to chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
        return ripeness_message
    else:
        remaining_items = [item for item in pred_classes_names if item != "Tomato"]
        return f"No tomatoes detected. Items in the fridge: {', '.join(remaining_items)}"

def main():
    st.markdown(
        "<h1 style='text-align: center;'>Welcome back</h1>",
        unsafe_allow_html=True,
    )
    temperature = st.number_input(
        "Temperature (°C)", value=8.0, min_value=-10.0, max_value=15.0, step=1.0
    )
    humidity = st.number_input(
        "Humidity (%)", value=75.0, min_value=0.0, max_value=100.0, step=1.0
    )

    st.markdown(
        "<h3 style='text-align: center;'>What would you like to do?</h3>",
        unsafe_allow_html=True,
    )

    if "selected_image" not in st.session_state:
        simulation_image_path = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/simulation"
        image_files = glob.glob(os.path.join(simulation_image_path, "*.[jp][pn]g"))
        if not image_files:
            st.error("No images found in the test directory.")
            st.stop()
        st.session_state.selected_image = random.choice(image_files)

    selected_image = st.session_state.selected_image

    #st.image(selected_image, caption="This is what you've got inside your refrigerator", use_container_width=True)

    if st.button("See what's inside"):
        predictor, metadata = setup_faster_rcnn()
        im = cv2.imread(selected_image)

        outputs = predictor(im)

        v = Visualizer(
            im[:, :, ::-1],
            metadata=metadata,
            scale=0.8
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

        processed_img_path = "fridge_analysis.png"
        cv2.imwrite(processed_img_path, out.get_image()[:, :, ::-1])

        st.success("Analysis Complete!")
        st.image(processed_img_path, caption="Detected Items in the Fridge", use_container_width=True)

        pred_classes_indices = outputs["instances"].pred_classes.cpu().numpy()
        pred_classes_names = [metadata.thing_classes[i] for i in pred_classes_indices]
        item_counts = {item: pred_classes_names.count(item) for item in set(pred_classes_names)}

        st.write("Contents of the fridge:")
        for item, count in item_counts.items():
            st.write(f"- {count} {item}(s)")

    if st.button("See the maturity"):
        predictor, metadata = setup_faster_rcnn()
        yolo_model = setup_yolo()

        img = cv2.imread(selected_image)

        results = yolo_model([img])

        for result in results:
            result.save(filename="output.jpg")

        ripeness_analysis = analyze_image(selected_image, temperature, humidity, predictor, metadata, yolo_model)

        st.success("Analysis Complete!")
        st.image("output.jpg", caption="Ripeness of tomatoes detected", use_container_width=True)

        if isinstance(ripeness_analysis, list):
            st.write("Ripeness Results:")
            for msg in ripeness_analysis:
                st.write("- " + msg)
        else:
            st.write(ripeness_analysis)

if __name__ == "__main__":
    main()

In [ ]:
from pyngrok import ngrok
import os

os.system("streamlit run smart_fridge_app.py &")

public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")